In [2]:
import pandas as pd
import os

In [5]:
files = [os.path.join('raw_state_county_data', file) for file in os.listdir('raw_state_county_data') if file.endswith('xlsx')]

In [72]:
state_data_list = list()

cols = {'State': 'state',
        'County': 'county',
        ('Diabetes prevalence', '% Adults with Diabetes'): 'diabetes_rate',
        ('Food insecurity', '% Food Insecure'): 'food_insecurity',
        ('Children eligible for free or reduced price lunch', '% Enrolled in Free or Reduced Lunch'): 'free_lunch'}

for file in files[:]:
    county_data = pd.read_excel(file, sheet_name='Additional Measure Data', header=[0, 1])
    new_cols = [col_tup if not any('Unnamed' in col for col in col_tup) else col_tup[1] for col_tup in county_data.columns]
    county_data.columns = new_cols

    try:
        county_data['County'] = county_data['County'].fillna('')
    except KeyError:
        try:
            county_data['County'] = county_data['Parish'].fillna('')
        except KeyError:
            county_data['County'] = county_data['Borough'].fillna('')
    
    state_data = county_data[cols.keys()]
    state_data = state_data.rename(columns=cols).set_index(['state', 'county'])

    state_data.fillna(state_data.iloc[0])
    
    state_data_list.append(state_data)
    

In [73]:
final = pd.concat(state_data_list)

In [74]:
final

diabetes_rate  food_insecurity  free_lunch
state        county                                              
Hawaii                          10.3             11.3   47.124759
             Hawaii             10.3             11.8   63.334669
             Honolulu           10.4             10.5   43.225504
             Kalawao            10.2             17.8         NaN
             Kauai               9.7              9.7   47.585227
...                              ...              ...         ...
North Dakota Traill             13.0              5.1   24.583031
             Walsh               9.9              5.3   37.486218
             Ward                8.8              7.1   27.825233
             Wells               9.7              5.1   29.251701
             Williams            9.4              5.8   28.699089

[3193 rows x 3 columns]